# Initialization

In [1]:
print("Hello there.")

Hello there.


In [2]:
# Colab specific data upload
!unzip cran.zip

Archive:  cran.zip
replace cran/cran.all.1400? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
# Auto re-import .py files
%load_ext autoreload
%autoreload 2

# Plotting with plt
%matplotlib inline

In [4]:
import numpy as np
import pandas as pd
import random

from collections import defaultdict
from sklearn.model_selection import train_test_split

from preprocessing import *
from synonym_enrich import *
from cosine_sim import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
!pip install --upgrade gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (4.0.1)


/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Code for `base.py`


In [6]:
def create_term_doc_matrix(base_docs):
    """ Constructs a frequency term-document matrix
    
    this function takes in a list of documents and returns a term-document matrix
    the rows are lemma types, the columns are documents 
    the rows should be sorted alphabetically
    the order of the columns should be preserved as it's given in base_docs
    the cell values are a number of times a lemma was seen in a document
    the value should be zero, if a lemma is absent from a document
    
    Parameters
    ----------
    base_docs : a list of lists of strings [['a','a','b'], ['a','b','c']]
        a list of documents represented as a list of lemmas
    
    Returns
    -------
    matrix : numpy array
        a matrix where columns are documents and rows are lemma types,
        the cells of the matrix contain lemma counts in a document,
        the lemmas for rows are sorted alphabetically
        for the example above it will be:
            np.array([[2,1],
                      [1,1],
                      [0,1]])
        
    sorted_vocab : list of strings
        a list of all the lemma types used in all documents (the rows of our matrix)
        the words should be strings sorted alphabetically
        for the example above it should be ['a','b','c']
    """
    sorted_vocab = list({word for doc in base_docs for word in set(doc)})    
    sorted_vocab.sort()

    freqs = []
    for doc in base_docs:
        counter = {k: 0 for k in sorted_vocab}
        for k, v in Counter(doc).items():
            counter[k] = v
        freqs.append(list(counter.values()))
    
    return np.array(freqs, dtype=np.float64).T, sorted_vocab


def create_term_doc_matrix_queries(normalized_queries, sorted_vocabulary):
    """ Constructs a frequency term-document matrix for queries
    
    this function takes in a list of queries and a vocabulary list and returns a term-document matrix
    the rows are lemma types as given in vocabulary, the columns are documents
    the rows should be in the same order as in vocabulary given
    the order of the columns should be preserved as it's given in normalized_queries
    the cell values are a number of times a lemma was seen in a document
    the value should be zero, if a lemma is absent from a document
    
    Parameters
    ----------
    normalized_queries : a list of lists of strings [['a','a','b','d'], ['a','b','c']]
        a list of queries represented as a list of lemmas
    sorted_vocabulary : list of strings
        a list of all the lemma types used in all training documents (the rows of our matrix)
        the words are strings sorted alphabetically
        for our example it will be ['a','b','c']
    
    Returns
    -------
    query_matrix : numpy array
        a matrix where columns are documents in normalized_queries 
        and rows are lemma types from sorted_vocabulary.
        for the example above it will be:
            np.array([[2,1],
                      [1,1],
                      [0,1]])
        'd' is not included in the matrix, because it is absent from sorted_vocabulary
    """
    filtered_queries = [list(filter(lambda x: x in sorted_vocabulary, doc)) for doc in normalized_queries]

    freqs = []
    for song in filtered_queries:
        counter = {k: 0 for k in sorted_vocabulary}
        for k, v in Counter(song).items():
            counter[k] = v
        freqs.append(list(counter.values()))
    
    return np.array(freqs, dtype=np.float64).T


def tf_idf(td_matrix):
    """ Weighs a term-document matrix of raw counts with tf-idf scheme
    
    this function takes in a term-document matrix as a numpy array, 
    and weights the scores with the tf-idf algorithm described above.
    idf values are modified with log_10
    
    Parameters
    ----------
    td_matrix : numpy array 
        a matrix where columns are songs and 
        rows are word counts in a song
    
    Returns
    -------
    tf_idf_matrix : numpy array 
        a matrix where columns are songs and 
        rows are word tf-idf values in a song
        
    idf_vector : numpy array of shape (vocabulary-size, 1)
        a vector of idf values for words in the collection. the shape is (vocabulary-size, 1)
        this vector will be used to weight new query documents
    """
    df_vector = np.sum(np.sign(td_matrix), axis=1)[..., np.newaxis]
    n = td_matrix.shape[1]
    idf_vector = np.log10(n / df_vector)
    
    return td_matrix * idf_vector, idf_vector


def lsi(matrix, d):
    """ Returns truncted SVD components
    
    this function takes in a term-document matrix, where
    values can be both raw frequencies and weighted values (tf_idf, ppmi)
    and returns their trunctaded SVD matrices.


    Parameters
    ----------
    matrix : numpy array
        a numpy array where columns are songs and 
        rows are lemmas
    d : int
        a number of features we will be reducing our matrix to
    
    Returns
    -------
    DT_d : numpy array
        a [d x m], where m is the number of word dimensions in the original matrix, 
        and d is the number of features we want to keep
        this is a matrix that represents documents with values for d hidden topics
    transformation_matrix : numpy array 
        a matrix to transform queries into the same vector space as DT_d
        T_dS_d^-, where S_d^- is inverse of S_d
    """

    T, s, DT = np.linalg.svd(matrix)
    return DT[:d, :], T[:, :d] @ np.linalg.inv(np.diag(s[:d]))


def read_cran_relevancy(path, relevancy_threshold=3):
    """ Loads query relevancy information from the cran dataset.

    Parameters
    ----------
    path : string
        path to the cranqrel file containing space-separated columns:
            query_id relevant_document_id relevant_document_number

    relevancy_threshold : int
        relevant_document_number column values are in 5 classes,
        1 - most important, 5 - no relevance
        this argument defines up to which relevancy the queries are paired
    
    Returns
    -------
    query_results : {qid: list of results sorted by relevancy, then by document id} 
    """

    # TODO: Make some sanity check.
    # Here, indexes should correspond, since
    #  td_docs, sorted_vocab = create_term_doc_matrix(docs)
    #  td_queries = create_term_doc_matrix_queries(queries, sorted_vocab)
    # so queries are indexed the same way as docs.
    # In addition, indexes from create_term_doc_matrix should correpond to
    # indexes from cran.all.1400, since documents are indexed one-by-one.

    query_results = defaultdict(list)

    with open(path) as f:
        for line in f.readlines():
            qid, docid, relevancy = tuple(line.split())
            qid, docid, relevancy = int(qid), int(docid), int(relevancy)
            if relevancy <= relevancy_threshold:
                query_results[qid].append((docid, relevancy))
    
    for k, v in query_results.items():
        query_results[k] = [docid for (docid, relevancy) in sorted(v, key=lambda x: x[1])]
    
    return dict(query_results)

# read_cran_relevancy("cran/cranqrel")

# Code for `validate.py`

In [7]:
def closest_n_documents(matrix_collection, matrix_queries, n):
    """Finds N closest documents from a training collection to every song in a test collection
    
    this function takes in original document collection, new document collection,
    computes cosine similarity between documents in old and new collection, 
    and outputs the list of n-closest documents to each new song
    when a vector in a query has only zeros, 
        the closeness to it should be determined by index of a song from a matrix_collection:
        the closest document for a zero vector has index 0
        the second closest document for a zero vector has index 1 and so on
    
    Parameters
    ----------
    matrix_collection : numpy array
        a term-document matrix of songs in training collection
        songs are columns
    matrix_queries : numpy array
        a term-document matrix of query songs
        songs are columns
    n : int
        a number of closest documents to return
    
    Returns
    -------
    closest_docs : a list of lists 
        a list of length equal to the number of songs in a query matrix
        each element is, in turn, a list of n imdices of documents in matrix_collection that were closest to the query
        for n=2 and query matrix with 3 songs, the out put should look like so [[1,2],[1,2],[1,2]]
    """
    _, n1 = matrix_collection.shape
    _, n2 = matrix_queries.shape
    
    closest_docs = []
    for i_test in range(n2):
        query = matrix_queries[:, i_test]
        
        if not np.any(query):
            closest_docs.append(list(range(n)))
            continue
        
        cosines = np.zeros(n1)
        for i_train in range(n1):
            cosines[i_train] = cosine_sim(matrix_collection[:, i_train], query)
        top_idxs = cosines.argsort()[-n:][::-1]
        closest_docs.append(top_idxs.tolist())

    return closest_docs


def get_k_relevant(k, query, D):
    """returns ranked list of top k documents in descending order of their
    cosine similarity with the query
        
    Parameters
    ----------
    k : int
        number of documents to retrieve (top k)
    query : numpy.ndarray
        vector representation of query whose cosine similarity is to be computed with the corpus
    D: list of numpy.ndarray
        vector representation of all documents in corpus
    
    Returns
    -------
    ranked_sims: list of tuples (cosine similarity, index of document)
        list of top k cosine similarities and the corresponding documents (their index) in descending order
    """
      
    cosine_sims = []
    
    for i, d in enumerate(D):
        cosine_sims.append((cosine_sim(query, d), i))
        
    ranked_sims = sorted(cosine_sims, key=lambda x: x[0], reverse=True)
    
    if k != 0:
        # if k=0 retrieve all documents in descending order
        ranked_sims = ranked_sims[:k]
    
    return ranked_sims

# Retrieval Metrics,
# patched version from https://github.com/lgalke/vec4ir/blob/master/vec4ir/rank_metrics.py

def precision(r):
    r = np.asarray(r)
    if r.size == 0:
        return 0
    tp = np.count_nonzero(r)
    return tp / r.size

def recall(r, n_relevant):
    return np.count_nonzero(r) / n_relevant

def precision_at_k(r, k):
    """Score is precision @ k

    Relevance is binary (nonzero is relevant).

    >>> r = [0, 0, 1]
    >>> precision_at_k(r, 1)
    0.0
    >>> precision_at_k(r, 2)
    0.0
    >>> precision_at_k(r, 3)
    0.33333333333333331
    >>> precision_at_k(r, 4)
    Traceback (most recent call last):
        File "<stdin>", line 1, in ?
    ValueError: Relevance score length < k


    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Precision @ k

    Raises:
        ValueError: len(r) must be >= k
    """
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):
    """Score is average precision (area under PR curve)

    Relevance is binary (nonzero is relevant).

    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1]
    >>> delta_r = 1. / sum(r)
    >>> sum([sum(r[:x + 1]) / (x + 1.) * delta_r for x, y in enumerate(r) if y])
    0.7833333333333333
    >>> average_precision(r)
    0.78333333333333333
    >>> r = [1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0]
    >>> average_precision(r)
    0.78333333333333333
    >>> average_precision([1,1,0,0]) == average_precision([1,1])
    True
    >>> average_precision([0])
    0.0


    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.0
    return np.mean(out)

def mean_average_precision(rs):
    """Score is mean average precision

    Relevance is binary (nonzero is relevant).

    >>> rs = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1]]
    >>> mean_average_precision(rs)
    0.78333333333333333
    >>> rs = [[1, 1, 0, 1, 0, 1, 0, 0, 0, 1], [0]]
    >>> mean_average_precision(rs)
    0.39166666666666666

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Mean average precision
    """
    return np.mean([average_precision(r) for r in rs])


def preprocess(docs, preprocessings):
    for fnt in preprocessings:
        docs = fnt(docs)
    return docs

def query(qids, queries, sorted_vocab):
    """ Transforms queries given by qids into their matrix representation.

    Parameters
    ----------
    qids : list of ints
        query ids to transform
    queries : list of strings
        list of query lemmas
    sorted_vocab: list of strings
        a list of all the lemma types used in all documents (the rows of our matrix)
    
    Returns
    -------
    query_matrix : numpy array
        a matrix where columns are documents in normalized_queries 
        and rows are lemma types from sorted_vocabulary.
    """
    # TODO: add embedding option
    queries = [queries[i] for i in qids]
    return create_term_doc_matrix_queries(queries, sorted_vocab)

def evaluate(td_docs, td_queries, qids_train, qids_test, n_relevant):
    pass

# Validation Core

In [8]:
# Set random seed to make experiments reproducible
random.seed(0)

In [9]:
# Parameters
data_parse_fnt = parseDocs
retrieval_fnt = read_cran_relevancy
doc_args = ["cran/cran.all.1400",]
query_args = ["cran/cran.qry",]
retrieval_args = ["cran/cranqrel",]
preprocessings = [
    tokenize_and_clean,
    lemmatize,
]
n_relevant = 10
test_size = 0.3


# Load documents
docs = data_parse_fnt(*doc_args)

# Load queries
queries = data_parse_fnt(*query_args)
query_results = retrieval_fnt(*retrieval_args)

# Split train, val queries:
qids_train, qids_test = train_test_split(list(range(1, len(queries))), test_size=test_size)

# Preprocess + make model for documents
docs = preprocess(docs, preprocessings)  # list of list of lemmatized tokens
td_docs, sorted_vocab = create_term_doc_matrix(docs)
queries = preprocess(queries, preprocessings)
# td_queries = create_term_doc_matrix_queries(queries, sorted_vocab)

# tf-idf optional stuff
# tf_idf_docs, idf_vector = tf_idf(td_docs)
# tf_idf_queries = td_queries * idf_vector

# Dimensionality reduction optional stuff
# td_docs_dense, td_docs_transform = lsi(tf_idf_docs, 2)
# tf_idf_docs_dense, tf_idf_docs_transform = lsi(tf_idf_docs, 2)
# td_queries_dense = td_queries.T.dot(td_docs_transform).T
# tf_idf_queries_dense = tf_idf_queries.T.dot(tf_idf_docs_transform).T

# TODO: add this to the retrieval process
# Output like this: embedding.wv['computer']
embedding = Word2Vec.load("w2v.model")

# def evaluate(td_docs, td_queries, qids_train, qids_test, n_relevant)

results = {}
rs = []

for qid in qids_test:
    ranked_sims = get_k_relevant(n_relevant, query([qid], queries, sorted_vocab), td_docs)
    docs_pred = [x[1] for x in ranked_sims]
    docs_true = query_results[qid][:n_relevant]
    rs.append([int(x[0] == x[1]) for x in zip(docs_pred, docs_true)])

results["MAP"] = mean_average_precision(rs)

ValueError: ignored

In [ ]:
embedding.wv['computer']

# Other code

Just for inspiration.

In [ ]:
def compute_average_results(closest_songs, train_index, test_index):
    """Computes average metrics for a model based on n closest songs for a test set 
    
    Parameters
    ----------
    closest_songs : a list of lists 
        a list of length equal to the number of songs in a query matrix
        each element contain n closest songs from a training collection to that song
    train_index : dict {atrist:lits of song indices}
        indices of songs in training collection assigned to artists
    test_index : dict {atrist:lits of song indices}
        indices of songs in test collection assigned to artists
    
    Returns
    -------
    average_precision: float 
        average precision based on all test songs
    average_recall: float
        average recall based on all test songs
    average_accuracy: float
        average accuracy based on all test songs
    average_error: float
        average error based on all test songs
    average_f_measure: float
        average f_measure based on all test songs
    """
    
    precision = recall = accuracy = error = f_measure = n_tests = 0
    n = sum([len(x) for x in train_index.values()])
    
    art_song_pairs = [(k, x) for k,v in test_index.items() for x in v]
    for artist, song_idx in art_song_pairs:
        if song_idx >= len(closest_songs):
            continue
            
        n_tests += 1
        q_result = closest_songs[song_idx]
        
        other_artists = list(train_index.keys())
        other_artists.remove(artist)
        tp = sum([x in train_index[artist] for x in q_result])
        fp = sum([x not in train_index[artist] for x in q_result])
        fn = sum([x not in q_result for x in train_index[artist]])
        tn = sum([x not in q_result for a in other_artists for x in train_index[a]])

        p = tp / (tp + fp)
        r = tp / (tp + fn)
        precision += p
        recall += r
        accuracy += (tp + tn) / n
        error += (fp + fn) / n
        if p + r > 0:
            f_measure += (2 * p * r) / (p + r)
    
    return precision / n_tests, recall / n_tests, accuracy / n_tests, error / n_tests, f_measure / n_tests